In [ ]:
import spacy
# import scispacy
# import en_ner_bc5cdr_md
from negspacy.negation import Negex
from spacy.matcher import Matcher

import numpy as np
import pandas as pd

In [ ]:
# nlp = spacy.load("en_ner_bc5cdr_md") 
# nlp.add_pipe("negex")


import numpy as np
import pandas as pd

In [ ]:
# load patient notes (unfiltered)

datafolder = '../data/'
filename = 'data_labels.xlsx'
# Load data
df_notes = pd.read_excel(datafolder + filename)
patient_ids = df_notes['Study ID'].unique()
print('Unique patients: ', len(patient_ids))
print('# notes:', len(df_notes))
display(df_notes.head())

# Join all notes of a type for each patient 
pnote_map = {}
for pid in patient_ids:
    pfilt = (df_notes['Study ID'] == pid) & (~pd.isna(df_notes['Note Text']))
    notetypes = df_notes.loc[pfilt, 'Note Type'].unique()
    pnote_map[pid] = {}
    for nt in notetypes:
        nt_filt = df_notes['Note Type'] == nt
        pnotes = df_notes.loc[pfilt & nt_filt, 'Note Text'].to_list()
        
        pnote_map[pid][nt] = ' . \n'.join(pnotes)

In [ ]:
# average number of notes per patient
num_notes_perPatient = df_notes.groupby('Study ID').size()
average_notes = np.average(num_notes_perPatient)
print(average_notes)
std_notes = np.std(num_notes_perPatient)
print(std_notes)

In [ ]:
##CRAMPING

# Load models
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("negex")
# Pattern matching in text for keywords
keywords = ['cramp', 'cramping', 'cramps', 'cramped', 'spasm']
patterns = []
for kw in keywords:
    patterns.append([{'LOWER': kw}])
matcher = Matcher(nlp.vocab)
matcher.add('CRAMP_PATTERN', patterns)
# Iterate over every patient in the notes map to search for cramping mentions
final = 0
patients_yes = []
patients_yes_bynt = {}
for pid in pnote_map:
    for nt in pnote_map[pid]:
        # Retrieve all notes for current patient and note_type
        pnotes = pnote_map[pid][nt]
        # Replace specific phrases 
        pnotes = pnotes.replace('[]	Excessive Cramps:', 'no')
        pnotes = pnotes.replace('[x]	Excessive Cramps: no','no')
        pnotes = pnotes.replace('[x]	Excessive Cramps:no','no')
        pnotes = pnotes.replace('[x]	Excessive Cramps:  no','no')
        pnotes = pnotes.replace('[x]	No Excessive Cramps', 'no')
        # Process notes with NLP pipeline
        doc = nlp(pnotes)
        matches = matcher(doc)
        # If any matches are found, increase the count and store patient info
        if len(matches) > 0:
            final = final + 1
            patients_yes.append(pid)
            
            if nt not in patients_yes_bynt:
                patients_yes_bynt[nt] = []
            patients_yes_bynt[nt].append(pid)

# Calculate the percentage of patients with cramping mentioned in their records
perc = (final/len(pnote_map))*100
# Print results
print("Total number patients with cramps found:",final) #returns 12
print("Total percent of patients with cramping noted at any point in EHR:", round(perc,2),"%")
print("Studyid of patients:", list(np.unique(patients_yes)))
print(patients_yes_bynt)

In [ ]:
# ##CRAMPING

# nlp = spacy.load("en_core_web_sm")
# nlp.add_pipe("negex")

# pid = 21
# pnotes = pnote_map[pid]

# pnotes = pnotes.replace('[]	Excessive Cramps:', 'no')
# pnotes = pnotes.replace('[x]	Excessive Cramps: no','no')
# pnotes = pnotes.replace('[x]	Excessive Cramps:no','no')
# pnotes = pnotes.replace('[x]	Excessive Cramps:  no','no')
# pnotes = pnotes.replace('[x]	No Excessive Cramps', 'no')
# doc = nlp(pnotes)

# matcher = Matcher(nlp.vocab)
# pattern = [{'LOWER': 'cramp'}]
# pattern1 = [{'LOWER': 'cramping'}]
# pattern2 = [{'LOWER': 'cramps'}]
# pattern3 = [{'LOWER': 'cramped'}]
# pattern4 = [{'LOWER': 'spasm'}]
# #cramp, spasm, muscle stiffness, muscle pain, muscle contraction, NOT heat, 
# #menstrual and stomach cramps, remove []Excessive cramps: and [x]Excessive cramps: no
# matcher.add('CRAMP_PATTERN', [pattern,pattern1,pattern2,pattern3,pattern4])
# matches = matcher(doc)

# print(matches)
   

In [ ]:
##DRY SKIN
# Load models
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("negex")
# Pattern matching in text for keywords
keywords = ['dry skin', 'scaly skin', 'cracked skin', 'eczema', 'drying skin', \
            'skin dryness', 'dry;skin', 'skin dry']
patterns = []
for kw in keywords:
    patterns.append([{'LOWER': kw}])
matcher = Matcher(nlp.vocab)
matcher.add('DRY_PATTERN', patterns)
# Iterate over every patient in the notes map to search for dry skin mentions
final = 0
patients_yes = []
patients_yes_bynt = {}
for pid in pnote_map:
    for nt in pnote_map[pid]:
        # Retrieve all notes for current patient and note_type
        pnotes = pnote_map[pid][nt]
        # Process notes with NLP pipeline
        doc = nlp(pnotes)
        matches = matcher(doc)
        # If any matches are found, increase the count and store patient info
        if len(matches) > 0:
            final = final + 1
            patients_yes.append(pid)
            
            if nt not in patients_yes_bynt:
                patients_yes_bynt[nt] = []
            patients_yes_bynt[nt].append(pid)
# Calculate the percentage of patients with dry skin mentioned in their records
perc = (final/len(pnotes))*100
# Display results
print("Total number patients with dry skin found:",final) #returns 0
print("Total percent of patients with dry skin noted at any point in EHR:", round(perc,2),"%")
print("Studyid of patients:",patients_yes)
print(patients_yes_bynt)

In [ ]:
##FATIGUE
# Load models
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("negex")
# Pattern matching in text for keywords
keywords = ['fatigue', 'tired', 'fatigued', 'lethargic', 'lack of energy', \
            'weakness', 'malaise', 'tiredness', 'weariness', 'lacking in energy', \
            'loss of energy', 'energy decreased', 'decreased energy']
patterns = []
for kw in keywords:
    patterns.append([{'LOWER': kw}])
matcher = Matcher(nlp.vocab)
matcher.add('FATIGUE_PATTERN', patterns)
# Iterate over every patient in the notes map to search for fatigue mentions
final = 0
patients_yes = []
patients_yes_bynt = {}
for pid in pnote_map:
    for nt in pnote_map[pid]:
        # Retrieve all notes for current patient and note_type
        pnotes = pnote_map[pid][nt]
        # Replace phrases
        pnotes = pnotes.replace('Retired', 'no')
        pnotes = pnotes.replace('[] 	Lethargic  Respiratory', 'no')
        pnotes = pnotes.replace('Energy:','no')
        # Process notes with NLP pipeline
        doc = nlp(pnotes)
        matches = matcher(doc)
        # If any matches are found, increase the count and store patient info
        if len(matches) > 0:
            final = final + 1
            patients_yes.append(pid)
            
            if nt not in patients_yes_bynt:
                patients_yes_bynt[nt] = []
            patients_yes_bynt[nt].append(pid)
# Calculate the percentage of patients with fatigue mentioned in their records
perc = (final/len(pnote_map))*100
# Display results
print("Total number patients with fatigue found:",final) #returns 1
print("Total percent of patients with fatigue noted at any point in EHR:", round(perc,2),"%")
print("Studyid of patients:", list(np.unique(patients_yes)))
print(patients_yes_bynt)

In [ ]:
# ##FATIGUE

# nlp = spacy.load("en_core_web_sm")
# nlp.add_pipe("negex")

# pid = 75
# pnotes = pnote_map[pid]

# pnotes = pnotes.replace('Retired', 'no')
# pnotes = pnotes.replace('[] 	Lethargic  Respiratory', 'no')
# pnotes = pnotes.replace('Energy:','no')
# doc = nlp(pnotes)

# matcher = Matcher(nlp.vocab)
# pattern = [{'LOWER': 'fatigue'}]
# pattern1 = [{'LOWER': 'tired'}]
# pattern2 = [{'LOWER': 'fatigued'}]
# pattern3 = [{'LOWER': 'lethargic'}]
# pattern4 = [{'LOWER': 'lack of energy'}]
# pattern5 = [{'LOWER': 'weakness'}]
# pattern6 = [{'LOWER': 'malaise'}]
# #fatigue, tired, lethargic, lack of energy, weakness, weariness, burnt out, exhaustion
# matcher.add('FATIGUE_PATTERN', [pattern,pattern1,pattern2,pattern3,pattern4,pattern5,pattern6])
# matches = matcher(doc)

# print(doc)
# for match_id,start,end in matches:
#     print(doc[start:end])

In [ ]:
##ITCHING
# Load models
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("negex")
# Pattern matching in text for keywords
keywords = ['itch', 'itching', 'itches', 'itched', 'itchy', 'pruritus', \
            'pruritis', 'Pruritic disorder', 'pruritic dermatitis', \
            'scratching skin', 'excessive scratching']
patterns = []
for kw in keywords:
    patterns.append([{'LOWER': kw}])
    
matcher = Matcher(nlp.vocab)
matcher.add('ITCH_PATTERN', patterns)
# Iterate over every patient in the notes map to search for itching mentions
final = 0
patients_yes = []
patients_yes_bynt = {}
for pid in pnote_map:
    for nt in pnote_map[pid]:
        # Retrieve all notes for current patient and note_type
        pnotes = pnote_map[pid][nt]
        # Process notes with NLP pipeline
        doc = nlp(pnotes)
        matches = matcher(doc)
        # If any matches are found, increase the count and store patient info
        if len(matches) > 0:
            final = final + 1
            patients_yes.append(pid)

            if nt not in patients_yes_bynt:
                patients_yes_bynt[nt] = []
            patients_yes_bynt[nt].append(pid)
# Calculate the percentage of patients with itching mentioned in their records   
perc = (final/len(pnote_map))*100
# Display results   
print("Total number patients with itching found:",final) #returns 14
print("Total percent of patients with itching noted at any point in EHR:", round(perc,2),"%")
print("Studyid of patients:", list(np.unique(patients_yes)))
print(patients_yes_bynt)
   

In [ ]:
# ##IDH 

# nlp = spacy.load("en_core_web_sm")
# nlp.add_pipe("negex")
    
# final = 0
# patients_yes = []
# patients_yes_bynt = {}
# for pid in pnote_map:
#     for nt in pnote_map[pid]:
#         pnotes = pnote_map[pid][nt]
#         pnotes = pnotes.replace('[x]	Intradialytic hypotension: no', 'no')
#         pnotes = pnotes.replace('[x]	Intradialytic hypotension:no', 'no')
#         pnotes = pnotes.replace('[]	Intradialytic hypotension:', 'no')
#         doc = nlp(pnotes)

#         matcher = Matcher(nlp.vocab)
#         pattern = [{'LOWER': 'idh'}]
#         pattern1 = [{'LOWER': 'intradialytic'},{'LOWER': 'hypotension'}]
#         pattern2 = [{'LOWER': 'low'},{'LOWER': 'bp'}]
#         pattern3 = [{'LOWER': 'low'},{'LOWER': 'blood'},{'LOWER': 'pressure'}]
#         matcher.add('IDH_PATTERN', [pattern,pattern1,pattern2,pattern3])
#         matches = matcher(doc)
#         if len(matches) > 0:
#             final = final + 1
#             patients_yes.append(pid)
            
#             if nt not in patients_yes_bynt:
#                 patients_yes_bynt[nt] = []
#             patients_yes_bynt[nt].append(pid)
        
# perc = (final/len(pnote_map))*100
        
# print("Total number patients with idh found:",final)
# print("Total percent of patients with idh noted at any point in EHR:", round(perc,2),"%")
# print("Studyid of patients:",patients_yes)
# print(patients_yes_bynt)
   

In [ ]:
##MUSCLE SORE
# Load models
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("negex")
# Pattern matching in text for keywords
keywords = ['soreness', 'tender', 'tenderness', 'Myalgia', 'Myodynia']
patterns = [[{'LOWER': 'muscle'},{'LOWER': 'sore'}], [{'LOWER': 'muscle'},{'LOWER': 'cramp'}],  \
            [{'LOWER': 'muscle'},{'LOWER': 'pain'}], [{'LOWER': 'muscle'},{'LOWER': 'discomfort'}], \
            [{'LOWER': 'muscle'},{'LOWER': 'ache'}], [{'LOWER': 'muscle'},{'LOWER': 'aches'}]]
for kw in keywords:
    patterns.append([{'LOWER': kw}])
    
matcher = Matcher(nlp.vocab)
matcher.add('MUSCLESORENESS_PATTERN', patterns)
# Iterate over every patient in the notes map to search for muscle soreness mentions
final = 0
patients_yes = []
patients_yes_bynt = {}
for pid in pnote_map:
    for nt in pnote_map[pid]:
        # Retrieve all notes for current patient and note_type
        pnotes = pnote_map[pid][nt]
        # Process notes with NLP pipeline
        doc = nlp(pnotes)
        matches = matcher(doc)
        # If any matches are found, increase the count and store patient info
        if len(matches) > 0:
            final = final + 1
            patients_yes.append(pid)
            
            if nt not in patients_yes_bynt:
                patients_yes_bynt[nt] = []
            patients_yes_bynt[nt].append(pid)
# Calculate the percentage of patients with muscle soreness mentioned in their records       
perc = (final/len(pnote_map))*100
# Display results  
print("Total number patients with muscle sore found:", final)
print("Total percent of patients with muscle sore noted at any point in EHR:", round(perc,2),"%")
print("Studyid of patients:", list(np.unique(patients_yes)))
print(patients_yes_bynt)


In [ ]:
# ##MUSCLE SORE

# nlp = spacy.load("en_core_web_sm")
# nlp.add_pipe("negex")

# pid = 66
# pnotes = pnote_map[pid]
# doc = nlp(pnotes)

# matcher = Matcher(nlp.vocab)
# pattern =  [{'LOWER': 'soreness'}]
# pattern1 = [{'LOWER': 'muscle'},{'LOWER': 'sore'}]
# pattern2 = [{'LOWER': 'muscle'},{'LOWER': 'cramp'}]
# pattern3 = [{'LOWER': 'tender'}]
# pattern4 = [{'LOWER': 'tenderness'}]
# matcher.add('IDH_PATTERN', [pattern,pattern1,pattern2,pattern3,pattern4])
# matches = matcher(doc)

# print(doc)
# for match_id,start,end in matches:
#     print(doc[start:end])